In [1]:
%matplotlib inline

# Document similarity methods

In this notebook, we illustrate different document similarity methods and use them to retrieve similar customer reviews. The method word movers' distance is illustrated. This method depends on word embeddings too, but also look at an 'individual words' viewpoint.

## Word movers' distance

WMD uses normalised word embeddings and Bag of words to calculate distance between sentences/ documents. It resolves the problem of synonyms between sentences. The intution behind the method is that we find the minimum "traveling distance" between documents, in other words the most efficient way to "move" the distribution of sentence/document 1 to the distribution of sentence/ document 2.

A good illustration is the two sentences below:
sentence_obama and sentence_president

In [2]:
# Image from https://vene.ro/images/wmd-obama.png
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from time import time

#img = mpimg.imread('/content/drive/MyDrive/Colab Notebooks/images/wmd-obama.png')
#imgplot = plt.imshow(img)
#plt.axis('off')
#plt.show()

In [3]:
# Initialize logging.
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentence_obama = 'Obama speaks to the media in Illinois'
sentence_president = 'The president greets the press in Chicago'

## Preprocessing
First, let's do some pre-processing, removing stopwords and punctuation. 

In [4]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

# Pre-processing a document.
from nltk import word_tokenize
download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

sentence_obama = preprocess(sentence_obama)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We will use the word vectors that are pre-trained and available from google. https://code.google.com/archive/p/word2vec/ They are hosted in gensim. These are from part of Google News dataset (about 100 billion words) with 300-dimensional vectors for 3 million words and phrases.

More options:
https://radimrehurek.com/gensim/models/word2vec.html

In [5]:
import gensim.downloader as api
from gensim.models import Word2Vec
model = api.load('word2vec-google-news-300')

C:\anaconda\envs\pyakc\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
2021-10-17 19:33:06,695 : INFO : Creating C:\Users\anirban/gensim-data


[--------------------------------------------------] 1.4% 23.4/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=-------------------------------------------------] 3.5% 58.5/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[================================================--] 96.8% 1609.1/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.6% 1655.5/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2021-10-17 19:36:19,595 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from C:\\Users\\anirban/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2021-10-17T19:36:19.595242', 'gensim': '4.0.1', 'python': '3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'load_word2vec_format'}


In [11]:
!pip install pyemd

  Using cached pyemd-0.5.1.tar.gz (91 kB)
  Running setup.py clean for pyemd
Failed to build pyemd
    Running setup.py install for pyemd: started
    Running setup.py install for pyemd: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\anaconda\envs\pyakc\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\anirban\\AppData\\Local\\Temp\\pip-install-pj0m0uqk\\pyemd\\setup.py'"'"'; __file__='"'"'C:\\Users\\anirban\\AppData\\Local\\Temp\\pip-install-pj0m0uqk\\pyemd\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\anirban\AppData\Local\Temp\pip-wheel-7fz78qfv'
       cwd: C:\Users\anirban\AppData\Local\Temp\pip-install-pj0m0uqk\pyemd\
  Complete output (11 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\pyemd
  copying pyemd\__about__.py -> build\lib.win-amd64-3.7\pyemd
  copying pyemd\__init__.py -> build\lib.win-amd64-3.7\pyemd
  running build_ext
  building 'pyemd.emd' ext

The WMD model is based on the word vectors, and can be obtained directly from the distance measure from the individual word vectors. Note the sentence on Obama and the president has a distance of 3.79. 
Note that similarity and distance are opposites. When the distance is high, the similarity is low. Vice versa is true as well.

In [10]:
distance = model.wmdistance(sentence_obama, sentence_president)
print('distance = %.4f' % distance)

ModuleNotFoundError: No module named 'pyemd'

Notice that in another case with a very different sentence talking about oranges, the distance is bigger (less similar) at 4.4

In [ ]:
sentence_orange = preprocess('Oranges are my favorite fruit')
distance = model.wmdistance(sentence_obama, sentence_orange)
print('distance = %.4f' % distance)

## Normalising WMD
 
In the WMD distance, sentences of different lenghts can increase the distance. This is due to different word vectors lengths of the sentences. To mitigate this, the document vectors of the sentences are each normalised to the same dimension. The normalisation in this case reduces the distance lengths.

In [ ]:
model.init_sims(replace=True)  # Normalizes the vectors in the word2vec class.

distance = model.wmdistance(sentence_obama, sentence_president)  # Compute WMD as normal.
print('distance: %r' % distance)

distance = model.wmdistance(sentence_obama, sentence_orange)
print('distance = %.4f' % distance)

## Building our own word vectors

Instead of using the word vectors from Google, we train our own word vectors in this yelp reviews case. This is in json format and in particular we look at 6 restaurant IDs. This reviews dataset is available from Kaggle. https://www.kaggle.com/yelp-dataset/yelp-dataset 
The following code reads in the reviews one by one into the wmd_corpus (only for the 6 restaurant IDs) and the w2v corpus (for training word vectors.)

As always, training a new word vector model is advantageous since this trained model will reflect the semantics of this reviews case. 

In [ ]:
start = time()

import json

# Review IDs of the restaurants.
ids = ['fWKvX83p0-ka4JS3dc6E5A', 'IjZ33sJrzXqU-0X6U8NwyA', 'IESLBzqUCLdSzSqm0eCSxQ',
      '1uJFq2r5QfJG_6ExMRCaGw', 'm2CKSsepBCoRYWxiRUsxAg', 'jJAIXA46pU1swYyRCdfXtQ']

w2v_corpus = []  # Documents to train word2vec on (all 6 restaurants).
wmd_corpus = []  # Documents to run queries against (only one restaurant).
documents = []  # wmd_corpus, with no pre-processing (so we can see the original documents).
with open('/content/drive/MyDrive/Colab Data/yelp_academic_dataset_review.json') as data_file: 
    for line in data_file:
        json_line = json.loads(line)
        if json_line['review_id'] not in ids:
            # Not one of the 6 restaurants.
            continue
         
        # Pre-process document.
        text = json_line['text']  # Extract text from JSON object.
        text = preprocess(text)
        # Add to corpus for training Word2Vec.
        w2v_corpus.append(text)

        if json_line['review_id'] in ids:
            # Add to corpus for similarity queries.
            wmd_corpus.append(text)
            documents.append(json_line['text'])

Train a new word vector model for the WMD similarity and then use it for the Word similarity model (WMD). 

In [ ]:
# Train Word2Vec on all the restaurants. 
n_model = Word2Vec(w2v_corpus, workers=3, size=100, min_count=1)

# Initialize WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 5
instance = WmdSimilarity(wmd_corpus, model, num_best=5)  # means the top 5 documents are retrieved
# possible sentences come from only the wmd_corpus.
# WMD based on the word2vec from w2c corpus.
# you must first build vocabulary before training the model --> usually due to empty w2v_corpus

Test it on one of the review sentences. 

In [ ]:
start = time()
sent = 'love the gyro plate. Rice is so good and I also dig their candy selection.'
query = preprocess(sent)
sims = instance[query]  # A query is simply a "look-up" in the similarity class.
print ('Cell took %.2f seconds to run.' %(time() - start))

Output the num_best sentences that are most similar to the query.

In [ ]:
# Print the query and the retrieved documents, together with their similarities.
print ('Query:', sent)
for i in range(num_best):
    print (i, '----')
    print (documents[sims[i][0]])
    print ('sim = %.4f' % sims[i][1])

**Workshop submission:**

Use the news.xls corpus to replace the yelp academic dataset to re-construct a word vector model. Let’s call this word vector model WV_News and the word vector model from Yelp reviews WV_Yelp. 


*   Get the most similar document for the following two sentences from two models (Yelp & WV_NEWS):


sentence_1: The pandemic is showing little sign of slowing down, with more than 10,000 new deaths recorded worldwide every day. according to an AFP tally.

Yelp doc:

WV_NEWS doc:


---


Sentence_2:The dollar's weakening is likely to last at least another six months as investors continue to shift to risky assets and higher returns from a Reuters poll.

Yelp doc:

WV_NEWs doc:



---



*   Comment on your findings. For eg. On the choice of words, the usage/ size of the corpus – all these can affect the results). Does a different word vector corpus affect also the similarity scores? Also, on the ‘extracted’ news sentence outcome. Does it look suitable?





